### Standalone Parseland Parsing Job

Reads new HTML records from `openalex.taxicab.taxicab_results`, calls the Parseland API to extract
structured metadata (authors, URLs, license, version, abstract), and appends results to
`openalex.parseland.parsed_pages`.

**input**: `openalex.taxicab.taxicab_results` (HTML records not yet in `parsed_pages`)

**process**: Parseland API on ECS (GET /parseland/{taxicab_id})

**output**: `openalex.parseland.parsed_pages`

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import datetime
from datetime import timezone
from pyspark.sql import functions as F
from pyspark.sql import types as T
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [ ]:
PARSELAND_URL = "http://parseland-load-balancer-667160048.us-east-1.elb.amazonaws.com/parseland"

In [ ]:
author_schema = T.StructType([
    T.StructField("name", T.StringType(), True),
    T.StructField("is_corresponding", T.BooleanType(), True),
    T.StructField("affiliations", T.ArrayType(
        T.StructType([
            T.StructField("name", T.StringType(), True)
        ])
    ), True)
])

url_schema = T.StructType([
    T.StructField("url", T.StringType(), True),
    T.StructField("content_type", T.StringType(), True)
])

parsed_pages_schema = T.StructType([
    T.StructField("taxicab_id", T.StringType(), True),
    T.StructField("url", T.StringType(), True),
    T.StructField("resolved_url", T.StringType(), True),
    T.StructField("native_id", T.StringType(), True),
    T.StructField("native_id_namespace", T.StringType(), True),
    T.StructField("authors", T.ArrayType(author_schema), True),
    T.StructField("urls", T.ArrayType(url_schema), True),
    T.StructField("license", T.StringType(), True),
    T.StructField("version", T.StringType(), True),
    T.StructField("abstract", T.StringType(), True),
    T.StructField("had_error", T.BooleanType(), True),
    T.StructField("parsed_date", T.TimestampType(), True)
])

In [ ]:
%sql
CREATE SCHEMA IF NOT EXISTS openalex.parseland

In [ ]:
%sql
CREATE TABLE IF NOT EXISTS openalex.parseland.parsed_pages (
  taxicab_id STRING,
  url STRING,
  resolved_url STRING,
  native_id STRING,
  native_id_namespace STRING,
  authors ARRAY<STRUCT<name: STRING, is_corresponding: BOOLEAN, affiliations: ARRAY<STRUCT<name: STRING>>>>,
  urls ARRAY<STRUCT<url: STRING, content_type: STRING>>,
  license STRING,
  version STRING,
  abstract STRING,
  had_error BOOLEAN,
  parsed_date TIMESTAMP
)
USING DELTA

In [ ]:
# Find new HTML records not yet parsed
taxicab_results = (
    spark.read.table("openalex.taxicab.taxicab_results")
    .filter(
        F.col("taxicab_id").isNotNull() &
        F.col("content_type").contains("html")
    )
    .select("taxicab_id", "url", "resolved_url", "native_id", "native_id_namespace")
)

already_parsed = (
    spark.read.table("openalex.parseland.parsed_pages")
    .select("taxicab_id")
)

to_parse = taxicab_results.join(already_parsed, "taxicab_id", "left_anti")

to_parse_count = to_parse.count()
print(f"Records to parse: {to_parse_count:,}")

if to_parse_count == 0:
    dbutils.notebook.exit("No new records to parse")

In [ ]:
to_parse_pd = to_parse.toPandas()
records = to_parse_pd.to_dict('records')
print(f"Collected {len(records):,} records for parsing")

In [ ]:
def create_session(max_workers):
    """Create a requests session with proper connection pooling."""
    session = requests.Session()
    retries = Retry(
        total=3,
        backoff_factor=0.5,
        status_forcelist=[500, 502, 503, 504],
        allowed_methods=["GET"]
    )
    adapter = HTTPAdapter(
        pool_connections=max_workers,
        pool_maxsize=max_workers,
        max_retries=retries
    )
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session


def call_parseland(taxicab_id, session, timeout=30):
    """Call Parseland API for a single taxicab_id."""
    try:
        response = session.get(f"{PARSELAND_URL}/{taxicab_id}", timeout=timeout)
        if response.status_code == 200:
            data = response.json()
            return {
                "authors": data.get("authors") or [],
                "urls": data.get("urls") or [],
                "license": str(data.get("license") or ""),
                "version": str(data.get("version") or ""),
                "abstract": str(data.get("abstract") or ""),
                "had_error": False
            }
        else:
            return {"authors": [], "urls": [], "license": "", "version": "", "abstract": "", "had_error": True}
    except Exception as e:
        return {"authors": [], "urls": [], "license": "", "version": "", "abstract": "", "had_error": True}


def process_record(record, session):
    """Process a single record: call parseland and combine with metadata."""
    result = call_parseland(record["taxicab_id"], session)
    return {
        "taxicab_id": record["taxicab_id"],
        "url": record["url"],
        "resolved_url": record["resolved_url"],
        "native_id": record["native_id"],
        "native_id_namespace": record["native_id_namespace"],
        **result
    }

In [ ]:
MAX_WORKERS = 48
BATCH_SIZE = 50000

session = create_session(MAX_WORKERS)
start_time = time.time()
total_written = 0
batch = []

print(f"Starting Parseland parsing with {MAX_WORKERS} workers")
print(f"Writing results in batches of {BATCH_SIZE:,}")

def flush_batch(batch):
    global total_written
    now = datetime.datetime.now(timezone.utc)
    for r in batch:
        r["parsed_date"] = now
    df = spark.createDataFrame(batch, schema=parsed_pages_schema)
    df.write.mode("append").format("delta").saveAsTable("openalex.parseland.parsed_pages")
    total_written += len(batch)
    elapsed = time.time() - start_time
    rate = total_written / elapsed if elapsed > 0 else 0
    print(f"  >> Flushed {len(batch):,} results ({total_written:,} total, {rate:.1f}/sec)")

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = {
        executor.submit(process_record, record, session): record
        for record in records
    }
    for count, future in enumerate(as_completed(futures), 1):
        if count % 5000 == 0:
            elapsed = time.time() - start_time
            rate = count / elapsed if elapsed > 0 else 0
            remaining = (len(records) - count) / rate / 60 if rate > 0 else 0
            print(f"Processed {count:,}/{len(records):,} ({rate:.1f}/sec, ~{remaining:.1f} min remaining)")
        batch.append(future.result())
        if len(batch) >= BATCH_SIZE:
            flush_batch(batch)
            batch = []

if batch:
    flush_batch(batch)

elapsed = time.time() - start_time
print(f"\nDone: {total_written:,} records parsed in {elapsed/60:.1f} min")